In [1]:
import tensorflow as tf
import os

class MyFashionMnist(object):
  def train(self):
    mnist = tf.keras.datasets.mnist

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.summary()

    print("Training...")
    
    model.fit(
        x_train, y_train, 
        epochs=3, 
        validation_split=0.2 
    ) 
    
    score = model.evaluate(x_test, y_test, batch_size=128, verbose=0)
    print('Test accuracy: ', score[1])

In [2]:
if __name__ == '__main__':
    if os.getenv('FAIRING_RUNTIME', None) is None:
        from kubeflow import fairing
        from kubeflow.fairing.kubernetes import utils as k8s_utils
        
        PRIVATE_REGISTRY = 'registry.kube-system.svc.cluster.local:30000'
        
        fairing.config.set_builder(
            'append',
            base_image = 'tensorflow/tensorflow',
            registry = PRIVATE_REGISTRY,
            image_name='my-04-notebook-single-file-fairing-job', 
            push=True
        )
        
        fairing.config.set_deployer(
            'job',
            namespace='myspace',
            pod_spec_mutators=[
                k8s_utils.get_resource_mutator(cpu=1, memory=5)]
        )
        
        fairing.config.run()
    else:
        remote_train = MyFashionMnist()
        remote_train.train()

[I 210314 19:20:31 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.converted_notebook.ConvertNotebookPreprocessor object at 0x7ff73ae7a7f0>
[I 210314 19:20:31 config:136] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7ff745c06048>
[I 210314 19:20:31 config:138] Using deployer: <kubeflow.fairing.deployers.job.job.Job object at 0x7ff747e49198>
[W 210314 19:20:31 append:50] Building image using Append builder...
[I 210314 19:20:31 base:107] Creating docker context: /tmp/fairing_context_dlf1u0jc
[I 210314 19:20:31 converted_notebook:127] Converting 04-notebook-single-file-fairing.ipynb to 04-notebook-single-file-fairing.py
[I 210314 19:20:31 docker_creds_:234] Loading Docker credentials for repository 'tensorflow/tensorflow'
[W 210314 19:20:31 append:54] Image successfully built in 0.6869012940005632s.
[W 210314 19:20:31 append:94] Pushing image registry.kube-system.svc.cluster.local:30000/my-04-notebook-single-file-fairing-job:5EFC7685..

2021-03-14 19:20:47.080923: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-03-14 19:20:47.080973: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
11493376/11490434 [==============================] - 0s 0us/step
2021-03-14 19:20:49.969797: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-14 19:20:49.970113: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-03-14 19:20:49.970170: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-03-14 19:20:49.970201: I tensorflow/stream_executor/cuda/cuda

[W 210314 19:21:23 job:173] Cleaning up job fairing-job-28qbs...
